In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [6]:
from crewai import Agent, Task, Crew
import os

In [ ]:
os.environ['AWS_SESSION_TOKEN'] = ''
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''

In [8]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [9]:
planner = Agent(
    role="Report Planner",
    goal="Plan a comprehensive and factually accurate report on {topic}",
    backstory="You're an expert editor specializing in the meticulous planning of reports. "
              "For the topic: {topic}, you gather essential information, create an initial outline, "
              "and organize the content framework. Your objective is to provide a clear, logical structure "
              "that guides the Researcher in conducting detailed research and filling in the content. "
              "Your planning ensures the report is thorough, informative, and helps the audience gain "
              "insight and make well-informed decisions.",
    allow_delegation=False,
	verbose=True
)

In [10]:
researcher = Agent(
    role="Researcher",
    goal="Conduct web research on {topic} and provide comprehensive and accurate content.",
    tools=[search_tool, scrape_tool],
    backstory="You are an experienced researcher tasked with gathering detailed, reliable information on {topic}. "
              "By utilizing web search and scraping tools, you collect comprehensive data and insights. "
              "Your research serves as the foundational content that the Content Writer will use to create an informed and engaging article. "
              "Your thorough investigation ensures the report is factual, current, and highly informative.",
    allow_delegation=False,
    verbose=True
)

In [11]:
outline_editor = Agent(
    role="Outline Editor",
    goal="Review and refine the content outline to ensure it is comprehensive and well-structured.",
    backstory="You are a skilled editor responsible for reviewing and enhancing the content outline. "
              "Based on the initial outline and the latest research results, your task is to ensure the outline "
              "is complete, logically organized, and accurately reflects the research findings. "
              "Your meticulous editing prepares the outline for the Content Writer to transform it "
              "into a comprehensive and engaging article.",
    allow_delegation=False,
    verbose=True
)

In [12]:
writer = Agent(
    role="Report Writer",
    goal="Write a comprehensive and factually accurate report on {topic}.",
    backstory="You are a skilled report writer tasked with creating a detailed and informative report on {topic}. "
              "Your work is based on extensive research and a well-structured outline provided by the Research Planner and Outline Editor. "
              "Your objective is to transform this information into a coherent, engaging, and insightful report. "
              "You ensure that the report is complete, logically organized, and highly informative, "
              "delivering valuable insights to the audience and supporting them in making well-informed decisions.",
    allow_delegation=False,
    verbose=True
)

In [13]:
report_editor = Agent(
    role="Editor",
    goal="Edit a given research report to align with the overall writing style to be coherent and consistent.",
    backstory="You are an experienced editor responsible for reviewing the research report written by the Report Writer. "
              "Your goal is to ensure that the report adheres to the organization's writing style, follows journalistic best practices, "
              "and provides balanced viewpoints when presenting opinions or assertions. "
              "You meticulously review the content for accuracy, clarity, and coherence, ensuring that the report is polished, professional, "
              "and avoids major controversial topics or opinions whenever possible.",
    allow_delegation=False,
    verbose=True
)

In [14]:
plan = Task(
    description=(
        "1. Utilize your expertise to prioritize the most relevant trends, key concepts, "
        "and important aspects related to {topic}.\n"
        "2. Create a detailed content outline that includes main topics, subtopics, "
        "and a logical structure for the research report.\n"
        "3. Ensure that the outline provides a clear framework that guides further research "
        "and writing efforts."
    ),
    expected_output="A comprehensive content outline document with main topics, subtopics, "
                    "and a well-structured framework for the research report.",
    agent=planner,
)

In [15]:
research = Task(
    description=(
        "1. Use your web search and scraping tools to gather detailed and reliable information on {topic}.\n"
        "2. Identify and collect data from authoritative sources, ensuring the information is accurate and current.\n"
        "3. Organize the collected information into a comprehensive research document, organized by key points and subtopics.\n"
        "4. Cite all sources properly to ensure the credibility and traceability of the information."
    ),
    expected_output="A comprehensive research document with detailed summaries of web content, organized by key points and subtopics, including proper citations.",
    agent=researcher,
)

In [16]:
edit_outline = Task(
    description=(
        "1. Review the initial content outline to understand its structure and key points.\n"
        "2. Incorporate the results of the research conducted by the research agent.\n"
        "3. Update the outline to ensure it comprehensively covers all key points and findings from the research.\n"
        "4. Verify the logical flow and organization of the sections to ensure coherence.\n"
        "5. Ensure the outline is clear, detailed, and sufficiently structured for the Content Writer to follow easily."
    ),
    expected_output="A refined content outline that comprehensively covers all key points, logically organized, and ready for the Content Writer.",
    agent=outline_editor,
)

In [17]:
write = Task(
    description=(
        "1. Use the content plan and refined outline to craft a comprehensive research report on {topic}.\n"
        "2. Ensure sections and subtitles are engaging and appropriately named.\n"
        "3. Structure the report with a captivating introduction, insightful body, and a summarizing conclusion.\n"
        "4. Cite appropriate sources as footnotes, utilizing the research document for accurate referencing.\n"
    ),
    expected_output="A well-written research report in markdown format, ready for publication, "
                    "with each section containing 2 or 3 paragraphs and all sources cited as footnotes.",
    agent=writer,
)

In [18]:
edit_report = Task(
    description=(
        "1. Review the research report for grammatical errors and correct them.\n"
        "2. Ensure all sections are well-structured and each section contains 2 or 3 paragraphs.\n"
        "3. Verify that all sources are properly cited as footnotes.\n"
        "4. Make any necessary adjustments to improve clarity, coherence, and readability."
    ),
    expected_output="A polished and well-written research report in markdown format, ready for publication, "
                    "with each section containing 2 or 3 paragraphs and all sources cited as footnotes.",
    agent=report_editor,
)

In [20]:
crew = Crew(
    agents=[planner, researcher, outline_editor, writer, report_editor],
    tasks=[plan, research, edit_outline, write, edit_report],
    verbose=True
)

In [21]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence in 2024"})

2024-10-22 10:49:44,367 - 129944911008384 - llm.py-llm:161 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
2024-10-22 10:49:44,394 - 129944911008384 - llm.py-llm:161 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
2024-10-22 10:49:44,417 - 129944911008384 - llm.py-llm:161 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable


# Agent: Report Planner
## Task: 1. Utilize your expertise to prioritize the most relevant trends, key concepts, and important aspects related to Artificial Intelligence in 2024.
2. Create a detailed content outline that includes main topics, subtopics, and a logical structure for the research report.
3. Ensure that the outline provides a clear framework that guides further research and writing efforts.
# Agent: Report Planner
## Task: 1. Utilize your expertise to prioritize the most relevant trends, key concepts, and important aspects related to Artificial Intelligence in 2024.
2. Create a detailed content outline that includes main topics, subtopics, and a logical structure for the research report.
3. Ensure that the outline provides a clear framework that guides further research and writing efforts.
# Agent: Report Planner
## Task: 1. Utilize your expertise to prioritize the most relevant trends, key concepts, and important aspects related to Artificial Intelligence in 2024.
2. Crea

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable